# A quantum solution to the Discrete Logarithm Problem

*Qiskit Summer Jam, North Carolina, 2020*

*Matthew Gregoire, Richard Howe, Guan-Wen Chou, Anton Nikulsin, and Jessica Horn*

## The Discrete Logarithm Problem

In general terms, let $G$ be a group, let $a$ be an element of $G$ of order $r$, and let $b$ be some power of $a$, so that $b = a^m$, for $0 \leq m < r$. The discrete logarithm problem is as follows: given $a$ and $b$, find the integer $m$. While $b$ may be easy to calculate given $a$ and $m$, there is no known efficient classical algorithm for finding $m$ given $a$ and $b$. This fact is the basis of many modern cryptosystems. In this project, we present a novel method for solving the discrete logarithm problem.

## Oracle-based Algorithm and the Half-Bit

The *half-bit* of $b$, denoted $HB_a(b)$, is defined as follows.

$$ HB_a(b) = \begin{cases}
0 & 0 \leq m < r/2 \\
1 & r/2 \leq m < r
\end{cases}
$$

Thus the half-bit of $b$ has a natural interpretation as the most significant bit of the binary expansion of $m$. In his 1988 thesis, Burton S. Kaliski Jr. proves that, given an oracle that is able to correctly predict the half-bit with probability at least $1/2 + \epsilon$, the discrete logarithm problem can be solved in polynomial time, by making several oracle queries on values closely related to $b$. Kaliski also presents the algorithm to solve this problem.

In 2017, Kaliski published a paper describing a quantum circuit to efficiently implement this oracle, which he calls a quantum "magic box." While Shor's algorithm depends on solving the entire discrete logarithm problem using one quantum circuit, implementing this oracle would only require the quantum processor to calculate one bit of the discrete logarithm at a time, potentially saving valuable computational resources.

## Our Goal $\newcommand{\ket}[1]{\left|{#1}\right\rangle}$

In this project, our first and foremost goal was to implement the quantum "magic box" in Qiskit. We also wanted to implement Kaliski's discrete logarithm algorithm, and test it on a simple example. We were able to complete both of these goals successfully.

## The Quantum "Magic Box"

Below is our code for the quantum "magic box" algorithm in its entirety. Here is a brief summary of the functions used:

* `invert(x, N)`: returns the number $x^{-1}$ such that $xx^{-1} = 1$ in mod $N$.
* `create_unitary(a, N)`: this quantum algorithm frequently uses unitary transformations $U_{a^x}$, which transform the state $\ket{x}\ket{y}$ into the state $\ket{x}\ket{a^x y}$. This function creates a unitary matrix to perform this transformation.
* `create_controlled_unitary(a, N)`: similar to `create_unitary`, but returns a matrix which performs $U_{a^x}$ controlled by the first qubit.
* `oracle(a, b, N)`: Builds a quantum circuit to calculate the half-bit of $b$, then runs the circuit $\ceil{8\pi}$ times

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.circuit.library import QFT
from qiskit.extensions import UnitaryGate
import numpy as np

# Calculates the multiplicative inverse of x mod N
# (the number y such that xy = 1 (mod N)) using
# the extended Euclidean algorithm.
def invert(x, N):
    q = [0, 0]
    r = [N, x]
    t = [0, 1]

    while r[-1] != 0:
        q.append(r[-2]//r[-1])
        r.append(r[-2] - (q[-1]*r[-1]))
        t.append(t[-2] - (q[-1]*t[-1]))
    
    return int(t[-2] % N)

# Returns a unitary matrix which has the effect of multiplying each
# input |x> by a in mod N, resulting in the state |ax>.
def create_controlled_unitary(a, N):
    dim = 2**int(np.ceil(np.log(N)/np.log(2)) + 1)
    U = np.zeros((dim, dim))
    # Generate a permutation of the multiplicative group of Z_N.
    for i in range(int(dim/2)):
        U[i,i] = 1
    for i in range(N):
        U[int(dim/2) + i, ((a*i) % N)+int(dim/2)] = 1
    # The remaining states are irrelevant.
    for i in range(N, int(dim/2)):
        U[int(dim/2) + i, int(dim/2) + i] = 1
    return U

def create_unitary(a, N):
    a = int(np.round(a) % N)
    dim = 2**int(np.ceil(np.log(N)/np.log(2)))
    U = np.zeros((dim, dim))
    # Generate a permutation of the multiplicative group of Z_N.
    for i in range(N):
        U[i, ((a*i) % N)] = 1
    # The remaining states are irrelevant.
    for i in range(N, dim):
        U[i, i] = 1
    return U

# b is some power of a, and the oracle outputs m,
# where b = a^m (mod N) with >50% probability.
# (this is where our main algorithm goes)
def oracle(a, b, N, verbose=False):

    # Calculate the order of a
    r = 1
    product = a
    while product != 1:
        product = (product * a) % N
        r += 1

    # Find number of bits(n) needed to store a value from 0 to N-1
    # and initialize 2 quantum registers of size n
    n = int(np.ceil(np.log(N)/np.log(2)))
    qr1, qr2 = QuantumRegister(n), QuantumRegister(n)
    cr1, cr2 = ClassicalRegister(n), ClassicalRegister(1)
    qc = QuantumCircuit(qr1, qr2, cr1, cr2)
    
    #Change second register to state |00...01>
    qc.x(qr2[n-1])
    
    #Add H gate to first register
    for i in range(n):
        qc.h(qr1[i])
    
    # We need log_2(n) different matrices U_(a^(2^x))
    for i in range(n):
        U = create_controlled_unitary(a**(2**(n-i)) % N, N)
        qubits = [qr1[i]] + [qr2[j] for j in range(n)]
        qc.iso(U, qubits, [])


    qc.append(QFT(n), [qr1[i] for i in range(n)])

    for i in range(n):
        qc.measure(qr1[i], cr1[i])
    
    # Now cr1 is in state y. We define k to be the closest integer to y*r / 2**n.
    # Reuse the first quantum register, because we don't need it anymore.
    for i in range(2**(n-1), 2**n):
        qc.x(qr1[0]).c_if(cr1, i)

    qc.h(qr1[0])

    qc.barrier()

    # I don't think there's any way to get the result of the measurement mid-circuit
    # in qiskit. So this is a stop-gap method for now.

    for y in range(2**n):
        k = int(np.round(y*r/(2**n))) % r
        kInv = bin(invert(k, r))[2:]

        # Pad kInv with initial zeros
        while len(kInv) < n:
            kInv = '0' + kInv

        if '1' in kInv:
            for i in range(len(kInv)):
                bit = int(kInv[i])
                if bit == 1:
                    # Apply U operation only if the value of cr1 is y.
                    U = create_unitary(b**(2**i) % N, N)
                    qc.iso(U, [qr2[i] for i in range(n)], []).c_if(cr1, y)
    
    qc.barrier()
    qc.rz(-np.pi/2 , qr1[0])
    qc.h(qr1[0])
    qc.measure(qr1[0], cr2[0])
    
    if verbose:
        print(qc.draw(output="text"))

    backend_name = 'qasm_simulator'
    shots = int(np.ceil(8*np.pi))
    backend = Aer.get_backend(backend_name)
    if verbose:
        print("Running circuit on", backend_name, "...")
    result = execute(qc, backend, shots=shots).result().get_counts(qc)

    if verbose:
        print(result)
    
    zeros_count = 0
    ones_count = 0

    for k in result.keys():
        half_bit = k[0]
        if half_bit == '0':
            zeros_count += result[k]
        else:
            ones_count += result[k]

    if verbose:
        print('Zeros:', zeros_count, '\tOnes:', ones_count)
    
    if zeros_count > ones_count:
        return 0
    else:
        return 1